## File Preparations

In [1]:
# Dependencies
import pandas as pd
import csv
import os

In [2]:
# Read in CSV
csvpath = os.path.join("Resources", "purchase_data.csv")
pymoli_df = pd.read_csv(csvpath) # For getting transaction-based data
pymoli_unique_df = pymoli_df.drop_duplicates("SN","first") # For getting user-based data
pymoli_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Functions

In [3]:
# Builds out gender dictionaries
def buildGenderDictionary(dc,df): # By Gender (not overall counts):
        # For "Gender Demographics"
    dc["Total Count"] = len(df["SN"].unique()) # Count of players
    dc["Percent of Players"] = formatAsPercent(dc["Total Count"]/buyer_count) # Percentage of total players
        
        # For "Purchasing Analysis - Gender"
    dc["Transactions"] = df["Price"].count() # Count of total transactions
    dc["Total Purchase Amount"] = df["Price"].sum() # Total Purchase Value
    dc["Average Purchase Amount"] = formatAsCurrency(df["Price"].mean()) # Average Purchase Price
    dc["Average Purchase Amount Per Person"] = formatAsCurrency(dc["Total Purchase Amount"]/dc["Total Count"]) # Average Purchase Total per Person
    dc["Percent of Total Revenue"] = dc["Total Purchase Amount"]/total_revenue # Percent of Revenue
    dc["Total Purchase Amount"] = formatAsCurrency(dc["Total Purchase Amount"]) # format after making necessary calculations
    dc["Percent of Total Revenue"] = formatAsPercent(dc["Percent of Total Revenue"])

In [4]:
def formatAsPercent(v):
    return "{0:.2f}%".format(v*10*10)

In [5]:
def formatAsCurrency(v):
    return "${:,.2f}".format(v)

## General Numbers

#### Purchasing Analysis - Overview

In [6]:
# Total buyers count
buyer_count = pymoli_unique_df["SN"].count()
buyer_count

576

In [7]:
# Initialize Purchasing Analysis Dictionary
purchasing_analysis_overall = {}

In [71]:
# Number of Unique Items
purchasing_analysis_overall["Unique Items Available"] = [len(pymoli_df["Item Name"].unique())]
#### value is bracketed so it will act as index when creating the dataframe

In [72]:
# Average Purchase Price
purchasing_analysis_overall["Average Price"] = formatAsCurrency(round(pymoli_df["Price"].mean(),2))

In [73]:
# Total Number of Purchases
purchasing_analysis_overall["Total Purchases"] = pymoli_df["Purchase ID"].count()

In [74]:
# Total Revenue
total_revenue = pymoli_df["Price"].sum()
purchasing_analysis_overall["Total Revenue"] = formatAsCurrency(total_revenue)

In [75]:
# Configure and output table
purchasing_analysis_overall_df = pd.DataFrame(purchasing_analysis_overall)
purchasing_analysis_overall_df.head()

,Unique Items Available,Average Price,Total Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


#### Gender Demographics - Overview

In [13]:
# Initialize gender dictionaries and individual subset dataframes
male = {}
male_df = pymoli_df.loc[pymoli_df["Gender"] == "Male", :]

female = {}
female_df = pymoli_df.loc[pymoli_df["Gender"] == "Female", :]

other = {}
other_df = pymoli_df.loc[pymoli_df["Gender"] == "Other / Non-Disclosed", :]

In [14]:
# These will act as row index in gender_demographics_df
male["Gender"] = ["Male"]
female["Gender"] = ["Female"]
other["Gender"] = ["Other / Non-Disclosed"]

In [15]:
# Build out dictionaries for each gender
buildGenderDictionary(male, male_df)
buildGenderDictionary(female, female_df)
buildGenderDictionary(other, other_df)

In [16]:
# Post-building-dictionary adjustments: collect the dictionaries and convert/combine them to a useable dataframe
gender_dicts = [male, female, other]
gender_convert_dict_to_df = [pd.DataFrame(dc) for dc in gender_dicts]
gender_df = pd.concat(gender_convert_dict_to_df).set_index("Gender")

In [17]:
# All data was added to the appropriate dictionaries by the buildDictionary() function
# Output table to only show the data applicable to Gender Demographics
gender_df[["Total Count","Percent of Players"]]

,Total Count,Percent of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


#### Gender Demographics - Purchasing Analysis

In [18]:
# All data was added to the appropriate dictionaries by the buildDictionary() function
# Output table to only show the data applicable to Purchasing Analysis - Gender
gender_df[[
    "Transactions", "Total Purchase Amount", "Average Purchase Amount",
    "Average Purchase Amount Per Person", "Percent of Total Revenue"
]]

,Transactions,Total Purchase Amount,Average Purchase Amount,Average Purchase Amount Per Person,Percent of Total Revenue
Gender,,,,,
Male,652,"$1,967.64",$3.02,$4.07,82.68%
Female,113,$361.94,$3.20,$4.47,15.21%
Other / Non-Disclosed,15,$50.19,$3.35,$4.56,2.11%


#### Age Demographics - Overview

In [19]:
# Get the basic dataframe (unique) based on ages
age_overview_raw_df = pymoli_unique_df.sort_values("Age")

In [20]:
# Age Bins
bin_values = [0, 9, 14, 19, 24, 29, 34, 39, pymoli_unique_df["Age"].max()]
bin_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [21]:
# Bin the Age Ranges
age_overview_raw_df["Age Range"] = pd.cut(age_overview_raw_df["Age"], bin_values, labels=bin_labels, include_lowest=True)

In [22]:
# Get counts of each bin and make a new DataFrame
age_overview_df = pd.DataFrame(age_overview_raw_df["Age Range"].value_counts())

In [23]:
# Rename the "Age Range" column and sort the index to get the rows in order
age_overview_df = age_overview_df.rename(columns={"Age Range": "Count per Age Group"})
age_overview_df = age_overview_df.sort_index()

In [24]:
# The percent of the total players by Age
age_overview_df["Percent of Total Players"] = [formatAsPercent(x) for x in age_overview_df["Count per Age Group"]/buyer_count]
age_overview_df

,Count per Age Group,Percent of Total Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


#### Age Demographics - Minor vs Adult

In [25]:
# Get the basic dataframe (unique) based on ages
age_minoradult_raw_df = pymoli_unique_df.sort_values("Age")

In [26]:
# Age Bins for minor vs. adult
bin_values2 = [0, 17, pymoli_unique_df["Age"].max()]
bin_labels2 = ["Minor (<18)", "Adult (>=18)"]

In [27]:
# Bin the Age Ranges
age_minoradult_raw_df["Age Range"] = pd.cut(age_minoradult_raw_df["Age"], bin_values2, labels=bin_labels2, include_lowest=True)

In [28]:
# Get counts of each bin and make a new DataFrame
age_minoradult_df = pd.DataFrame(age_minoradult_raw_df["Age Range"].value_counts())

In [29]:
# Rename the "Age Range" column and sort the index to get the rows in order
age_minoradult_df = age_minoradult_df.rename(columns={"Age Range": "Count per Age Group"})
age_minoradult_df = age_minoradult_df.sort_index()

In [30]:
# The percent of the total players by Age
age_minoradult_df["Percent of Total Players"] = [formatAsPercent(x) for x in age_minoradult_df["Count per Age Group"]/buyer_count]
age_minoradult_df

,Count per Age Group,Percent of Total Players
Minor (<18),108,18.75%
Adult (>=18),468,81.25%


#### Age Demographics - Purchasing Analysis

In [31]:
# Get the basic dataframe (all) based on ages
age_purchase_raw_df = pymoli_df.sort_values("Age")

In [32]:
# Bin the Age Ranges
age_purchase_raw_df["Age Range"] = pd.cut(age_purchase_raw_df["Age"], bin_values, labels=bin_labels, include_lowest=True)

In [33]:
# Get counts of each bin and make a new DataFrame
age_purchase_df = pd.DataFrame(age_purchase_raw_df["Age Range"].value_counts())

In [34]:
# Rename the "Age Range" column and sort the index to get the rows in order
age_purchase_df = age_purchase_df.rename(columns={"Age Range": "Total Transactions"})
age_purchase_df = age_purchase_df.sort_index()

In [35]:
# Total Purchase Value
total_purchases = [age_purchase_raw_df.loc[age_purchase_raw_df["Age Range"] == x, :]["Price"].sum() for x in bin_labels]

In [36]:
# Average Purchase Price
avg_trans = [age_purchase_raw_df.loc[age_purchase_raw_df["Age Range"] == x, :]["Price"].mean() for x in bin_labels]

In [37]:
# Percentage of Total Revenue
perc_rev = [x/total_revenue for x in total_purchases]

In [38]:
# Average Purchase Total per Person
avg_per_person = [a/b for a,b in zip(total_purchases, age_overview_df["Count per Age Group"])]

In [39]:
# Format columns
age_purchase_df["Purchase Totals"] = [formatAsCurrency(x) for x in total_purchases]
age_purchase_df["Percentage of Total Revenue"] = [formatAsPercent(x) for x in perc_rev]
age_purchase_df["Average Transaction Amount"] = [formatAsCurrency(x) for x in avg_trans]
age_purchase_df["Average Transaction Per User"] = [formatAsCurrency(x) for x in avg_per_person]
age_purchase_df

,Total Transactions,Purchase Totals,Percentage of Total Revenue,Average Transaction Amount,Average Transaction Per User
<10,23,$77.13,3.24%,$3.35,$4.54
10-14,28,$82.78,3.48%,$2.96,$3.76
15-19,136,$412.89,17.35%,$3.04,$3.86
20-24,365,"$1,114.06",46.81%,$3.05,$4.32
25-29,101,$293.00,12.31%,$2.90,$3.81
30-34,73,$214.00,8.99%,$2.93,$4.12
35-39,41,$147.67,6.21%,$3.60,$4.76
40+,13,$38.24,1.61%,$2.94,$3.19


In [40]:
# Sorted table above
age_purchase_df["Average Transaction Per User"].sort_values(ascending=False)

35-39    $4.76
<10      $4.54
20-24    $4.32
30-34    $4.12
15-19    $3.86
25-29    $3.81
10-14    $3.76
40+      $3.19
Name: Average Transaction Per User, dtype: object

In [84]:
# Compare percent of purchasers against the revenue generated
compare_df = pd.DataFrame({})
compare_df["% of Total Players"] = age_overview_df["Percent of Total Players"]
compare_df["% of Revenue"] = age_purchase_df["Percentage of Total Revenue"]
compare_df["Delta"] = [str(round(float(a[:-1]) - float(b[:-1]),2))+"%" for a,b in zip(age_overview_df["Percent of Total Players"],age_purchase_df["Percentage of Total Revenue"])]
compare_df

,% of Total Players,% of Revenue,Delta
<10,2.95%,3.24%,-0.29%
10-14,3.82%,3.48%,0.34%
15-19,18.58%,17.35%,1.23%
20-24,44.79%,46.81%,-2.02%
25-29,13.37%,12.31%,1.06%
30-34,9.03%,8.99%,0.04%
35-39,5.38%,6.21%,-0.83%
40+,2.08%,1.61%,0.47%


## Superlatives

#### Top Spenders

In [42]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# SN
combined_SN_sum_df = pd.DataFrame(pymoli_df.groupby("SN").sum())
combined_SN_sum_df = combined_SN_sum_df.sort_values("Price", ascending=False)

In [43]:
# Total Purchase Value
top_spenders_df = pd.DataFrame({})
top_spenders_df["Total Purchase Value"] = combined_SN_sum_df.iloc[:,3]

In [44]:
# Transaction Count
combined_SN_count_df = pd.DataFrame(pymoli_df.groupby("SN").count())
top_spenders_df["Total Transactions"] = combined_SN_count_df.loc[top_spenders_df.index.values,["Purchase ID"]]

In [45]:
# Average Purchase Price
avg_trans_price = [a/b for a,b in zip(top_spenders_df["Total Purchase Value"], top_spenders_df["Total Transactions"])]
top_spenders_df["Average Transaction Price"] = avg_trans_price

In [46]:
#Format prices
top_spenders_df["Total Purchase Value"] = [formatAsCurrency(x) for x in top_spenders_df["Total Purchase Value"]]
top_spenders_df["Average Transaction Price"] = [formatAsCurrency(x) for x in top_spenders_df["Average Transaction Price"]]
top_spenders_df.head()

,Total Purchase Value,Total Transactions,Average Transaction Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


#### Most Popular Items

In [47]:
# Identify the 5 most popular items by purchase count, then list (in a table):
# Item ID
combined_item_count_df = pd.DataFrame(pymoli_df["Item ID"].value_counts())
combined_item_count_df = combined_item_count_df.sort_values("Item ID", ascending=False)

In [48]:
# Purchase Count
most_popular_df = pd.DataFrame({})
most_popular_df["Purchase Count"] = combined_item_count_df.iloc[:,0]
item_ids = most_popular_df.index # needed for getting Item Price and Item Name

In [49]:
# Item Price
item_prices = [pymoli_df.loc[pymoli_df["Item ID"] == item_id, "Price"].values[:].mean() for item_id in item_ids]
most_popular_df["Item Price"] = item_prices

In [50]:
# Total Purchase Value
popular_puchase_value = [a*b for a,b in zip(most_popular_df["Purchase Count"],item_prices)]
most_popular_df["Total Purchase Value"] = popular_puchase_value

In [51]:
# Item Name
item_names = [pymoli_df.loc[pymoli_df["Item ID"] == item_id, "Item Name"].values[0] for item_id in item_ids]
indices = pd.MultiIndex.from_arrays([item_ids, item_names], names=('Item ID', 'Item Name'))
most_popular_df = most_popular_df.set_index(indices)

In [52]:
# Before formatting, copy dataframe for use in "Greatest Revenue Generating Items"
greatest_revenue_df = most_popular_df.copy(deep=True)

In [53]:
#Format prices
most_popular_df["Item Price"] = [formatAsCurrency(x) for x in most_popular_df["Item Price"]]
most_popular_df["Total Purchase Value"] = [formatAsCurrency(x) for x in most_popular_df["Total Purchase Value"]]
most_popular_df.head(6)

### Diagram Note: I have included a 6th item (ID 82 - Nirvana) as it ties for 3rd at nine times purchased

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
132,Persuasion,9,$3.22,$28.99
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22


#### Greatest Revenue Generating Items

In [54]:
greatest_revenue_df = greatest_revenue_df.sort_values("Total Purchase Value", ascending=False)

In [55]:
#Format prices
greatest_revenue_df["Item Price"] = [formatAsCurrency(x) for x in greatest_revenue_df["Item Price"]]
greatest_revenue_df["Total Purchase Value"] = [formatAsCurrency(x) for x in greatest_revenue_df["Total Purchase Value"]]
greatest_revenue_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
